In [1]:
%matplotlib inline
from ipyparallel import Client, error
cluster=Client(profile="mpi")
view=cluster[:]
view.block=True

try:
    from openmdao.utils.notebook_utils import notebook_mode
except ImportError:
    !python -m pip install openmdao[notebooks]

OSError: You have attempted to connect to an IPython Cluster but no Controller could be found.
Please double-check your configuration and ensure that a cluster is running.

# Adding Constraints

To add a constraint to an optimization, use the `add_constraint` method on System.

```{eval-rst}
    .. automethod:: openmdao.core.system.System.add_constraint
        :noindex:
```

## Specifying units

You can specify units when adding a constraint. When this is done, the constraint value is converted from the target output’s units to the desired unit before giving it to the optimizer. If you also specify scaling, that scaling is applied after the unit conversion. Moreover, the upper and lower limits in the constraint definition should be specified using these units.



In [ ]:
import openmdao.api as om

prob = om.Problem()
model = prob.model

model.add_subsystem('comp1', om.ExecComp('y1 = 2.0*x',
                                         x={'val': 2.0, 'units': 'degF'},
                                         y1={'val': 2.0, 'units': 'degF'}),
                    promotes=['x', 'y1'])

model.add_subsystem('comp2', om.ExecComp('y2 = 3.0*x',
                                         x={'val': 2.0, 'units': 'degF'},
                                         y2={'val': 2.0, 'units': 'degF'}),
                    promotes=['x', 'y2'])

model.set_input_defaults('x', 35.0, units='degF')

model.add_design_var('x', units='degC', lower=0.0, upper=100.0)
model.add_constraint('y1', units='degC', lower=0.0, upper=100.0)
model.add_objective('y2', units='degC')

prob.setup()
prob.run_driver()

In [ ]:
print('Model variables')
print(prob.get_val('x', indices=[0]))

In [ ]:
print(prob.get_val('comp2.y2', indices=[0]))

In [ ]:
print(prob.get_val('comp1.y1', indices=[0]))

In [ ]:
print('Driver variables')
dv = prob.driver.get_design_var_values()
print(dv['x'][0])

In [ ]:
obj = prob.driver.get_objective_values(driver_scaling=True)
print(obj['y2'][0])

In [ ]:
con = prob.driver.get_constraint_values(driver_scaling=True)
print(con['y1'][0])

In [ ]:
import numpy as np
from openmdao.utils.assert_utils import assert_near_equal

assert_near_equal(prob.get_val('x', indices=[0]), 35.)
assert_near_equal(prob.get_val('y2', indices=[0]), 105.)
assert_near_equal(prob.get_val('y1', indices=[0]), 70.)
assert_near_equal(dv['x'][0], 1.6666666666666983)
assert_near_equal(obj['y2'][0], 40.555555555555586)
assert_near_equal(con['y1'][0], 21.111111111111143)

## Using the output of a distributed component as a constraint

You can use an output of a distributed component as a constraint or an objective. OpenMDAO automatically collects the values from all processors and provides them to the driver.

Here is an example where we perform optimization on a model that contains a `DistParabFeature` component that is distributed. The output is declared as a inequality constraint.

```{note}
This feature requires MPI, and may not be able to be run on Colab or Binder.
```

In [ ]:
import openmdao.api as om
om.display_source("openmdao.test_suite.components.paraboloid_distributed.DistParabFeature")

In [ ]:
%%px

import numpy as np
import openmdao.api as om

from openmdao.test_suite.components.paraboloid_distributed import DistParabFeature

size = 7

prob = om.Problem()
model = prob.model

ivc = om.IndepVarComp()
ivc.add_output('x', np.ones(size))
ivc.add_output('y', -1.42 * np.ones(size))

model.add_subsystem('p', ivc, promotes=['*'])
model.add_subsystem("parab", DistParabFeature(arr_size=size), promotes=['*'])

model.add_design_var('x', lower=-50.0, upper=50.0)
model.add_constraint('f_xy', lower=0.0)
model.add_objective('f_sum', index=-1)

prob.driver = om.pyOptSparseDriver(optimizer='SLSQP')
prob.setup()
        
prob.run_driver()

In [ ]:
%%px

desvar = prob.get_val('p.x', get_remote=True)
obj = prob.get_val('f_sum', get_remote=True)

print(desvar)

In [ ]:
%%px

print(obj)

In [ ]:
%%px

from openmdao.utils.assert_utils import assert_near_equal

assert_near_equal(prob.get_val('p.x'), np.array([2.65752672, 2.60433212, 2.51005989, 1.91021257, 1.3100763,  0.70992863, 0.10978096]), 1e-6 )
assert_near_equal(prob.get_val('f_sum'),11.50150011, 1e-6 )

## Adding multiple constraints on an array variable.

Sometimes you have a variable and you would like to constrain difference parts of it in different ways. OpenMDAO maintains the constraint data in a dictionary that keys on the variable pathname, so we need to specify an additional "alias" argument when we declare any additional constraints.

In the following example, the variable "exec.z" has an equality constraint on the first point and a lower bound on the end point. We constrain them by giving the second one an alias to differentiate it from the first one.

In [ ]:
p = om.Problem()

exec = om.ExecComp(['y = x**2',
                    'z = a + x**2'],
                a={'shape': (1,)},
                y={'shape': (101,)},
                x={'shape': (101,)},
                z={'shape': (101,)})

p.model.add_subsystem('exec', exec)

p.model.add_design_var('exec.a', lower=-1000, upper=1000)
p.model.add_objective('exec.y', index=50)
p.model.add_constraint('exec.z', indices=[0], equals=25)
p.model.add_constraint('exec.z', indices=[-1], lower=20, alias="End_Constraint")

p.driver = om.pyOptSparseDriver()
p.driver.options['optimizer'] = 'SLSQP'

p.setup()

p.set_val('exec.x', np.linspace(-10, 10, 101))

p.run_driver()

print(p.get_val('exec.z')[0], 25)
print(p.get_val('exec.z')[50], -75)

In [ ]:
assert_near_equal(p.get_val('exec.z')[0], 25)
assert_near_equal(p.get_val('exec.z')[50], -75)